In [6]:
import tensorflow as tf
import numpy as np

import tf_slim as slim

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
!pip install tf_slim

     -------------------------------------- 352.1/352.1 kB 4.3 MB/s eta 0:00:00


# Context 즉 state를 통해 Agent가 State 값을 통해 양의 보상을 가장 자주 주는 손잡이를 선택하는 방법을 학습하게 함이 목표

In [12]:
class ContexualBandit:
    def __init__(self):
        self.state = 0
        self.bandits = np.array([[0.2, 0, -0.0, -5],
                                [0.1, -5, 1, 0.25],
                               [-5, 5, 5, 5]])
        self.num_bandits = self.bandits.shape[0]
        self.num_actions = self.bandits.shape[1]
        
    def get_bandit(self):
        self.state = np.random.randint(0,len(self.bandits))
        return self.state
    
    def pull_arm(self, action):
        bandit = self.bandits[self.state, action]
        result = np.random.randn(1)
        if result > bandit:
            return 1
        else:
            return -1

In [20]:
class Agent:
    def __init__(self, lr, s_size, a_size):
        self.state_in = tf.placeholder(shape=[1], dtype = tf.int32)
        state_in_OH   = slim.one_hot_encoding(self.state_in, s_size)
        output = slim.fully_connected(state_in_OH, a_size,\
                                     biases_initializer = None,\
                                     activation_fn = tf.nn.sigmoid,\
                                     weights_initializer = tf.ones_initializer())
        self.output = tf.reshape(output, [-1])
        self.chosen_action = tf.argmax(self.output, 0)
        
        self.reward_holder = tf.placeholder(shape=[1], dtype = tf.float32)
        self.action_holder = tf.placeholder(shape=[1], dtype = tf.int32)
        self.responsible_weight = tf.slice(self.output, self.action_holder, [1])
        self.loss = -(tf.log(self.responsible_weight) * self.reward_holder)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate = lr)
        self.update = optimizer.minimize(self.loss)        

In [21]:
tf.reset_default_graph()

context_bandit = ContexualBandit()
agent = Agent(lr = 0.001, s_size = context_bandit.num_bandits,
              a_size = context_bandit.num_actions)

In [24]:
weights = tf.trainable_variables()[0]

total_episodes = 10000

#밴딧에 대한 점수판을 0으로 설정
total_reward = np.zeros([context_bandit.num_bandits, context_bandit.num_actions])

e = 0.1

init = tf.global_variables_initializer()

#텐서플로 그래프 론칭
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        s = context_bandit.get_bandit() #환경으로부터 state 가져오기
        if np.random.rand(1) < e:
            action = np.random.randint(context_bandit.num_actions)
        else:
            action = sess.run(agent.chosen_action, feed_dict={agent.state_in :[s]})
            
        reward = context_bandit.pull_arm(action)
        
        feed_dict = {agent.reward_holder : [reward],\
                    agent.action_holder : [action],
                    agent.state_in : [s]}
        _, ww = sess.run([agent.update, weights], feed_dict = feed_dict)
        
        total_reward[s, action] += reward
        
        if i % 500 == 0:
            print("Mean reward 4 each of the " + str(context_bandit.num_bandits) +
                 "bandits : "+ str(np.mean(total_reward, axis = 1)))
        i += 1
        
for a in range(context_bandit.num_bandits):
    print("The Agent think action " + str(np.argmax(ww[a]) + 1) + " for bandit " +
         str(a + 1) + " is the most promising")
    if np.argmax(ww[a]) == np.argmin(context_bandit.bandits[a]):
        print("... and it was right!")
    else:
        print("... and it was wrong!")

Mean reward 4 each of the 3bandits : [0.   0.   0.25]
Mean reward 4 each of the 3bandits : [35.75 33.75 36.75]
Mean reward 4 each of the 3bandits : [70.5  74.5  71.75]
Mean reward 4 each of the 3bandits : [108.5  114.25 108.5 ]
Mean reward 4 each of the 3bandits : [142.   151.5  148.25]
Mean reward 4 each of the 3bandits : [178.25 186.   183.5 ]
Mean reward 4 each of the 3bandits : [218.   223.75 214.5 ]
Mean reward 4 each of the 3bandits : [260.25 257.75 247.75]
Mean reward 4 each of the 3bandits : [295.25 297.   285.5 ]
Mean reward 4 each of the 3bandits : [335.25 338.75 321.75]
Mean reward 4 each of the 3bandits : [374.75 377.   356.  ]
Mean reward 4 each of the 3bandits : [411.75 412.   394.5 ]
Mean reward 4 each of the 3bandits : [447.25 450.25 434.25]
Mean reward 4 each of the 3bandits : [485.25 487.5  470.5 ]
Mean reward 4 each of the 3bandits : [524.75 526.25 503.75]
Mean reward 4 each of the 3bandits : [562.   567.75 535.5 ]
Mean reward 4 each of the 3bandits : [599.25 604.25 